# Extending PointNet networks and simulation with external and customized models.

## Example: Using Built-In NEURONAL Models

PointNet utlizes the NEST simulator for running point-neuron simulations, which includes many different varieties and types of neuronal models the majority of which can readily used during simulation in BMTK. An very useful practice is to take an existing network model built on model of a cell and replace it with a different cell model that more closely fits the needs of our research. With BMTK and SONATA this can be easily done, often without any programming, and even done on the fly before any given simulation.

A list of available models built into the NEST simulator can be found [here](https://nest-simulator.readthedocs.io/en/stable/models/index.html).

Most of the spiking models will work out-of-the-box in BMTK, readily swapped in and out of existing simulations, and you can use multiple models in the same network. (*We will ignore rates-based models which can only tentively work with BMTK. And for devices like genators and recorders PointNet uses a separate way of inserting them into the simulation*). To define what model will be used with a specific subset of cells and their properties, SONATA has two defined reserved attributes `model_template` and `dynamics_params` which we will take advantage of in this example.

### Setting the model type in our network.

The default way to set the model type for subset of neurons (which may be just a single neuron) that all use the same built-in NEST model type is to set the the **model_type** attributes to a value `nest:<nest-model-name>` where `<nest-model-name>` is the name of one of the spiking [NEST cell models](https://nest-simulator.readthedocs.io/en/stable/models/index.html). However, different models will have different parameters. There are multiple ways to set model parameters in BMTK, but the one that is most flexible and will make it easier to switch out different models while testing and optimizing our network is to set the **dynamics_params** to an easy to edit json file.

In the below example (*build_network.built_in.py*) will built a simple 100 cell network with synaptic stimuli. Initialally we will use [izhikevich](https://nest-simulator.readthedocs.io/en/stable/models/izhikevich.html) models using default params (**hint**: if the dynamics_params .json file is just an empty json file, then the cells will be loaded with the default params. This is a useful starting point). But you can change the **nest_model** (and optionally the **dynamics_params**) variable below to build the same network using different model types

In [1]:
nest_model = 'nest:izhikevich'
# nest_model = 'nest:iaf_psc_delta'
# nest_model = 'nest:aeif_cond_alpha'
# nest_model = 'nest:glif_psc'

dynamics_params = 'custom_model_params.default.json'
# dynamics_params = 'custom_model_params.izhikevich.json'
# dynamics_params = 'custom_model_params.aeif_cond_alpha.json'
# dynamics_params = 'custom_model_params.glif_psc.json'

In [2]:
import numpy as np
from bmtk.builder import NetworkBuilder


net = NetworkBuilder('net')
net.add_nodes(
    N=100,
    model_type='point_neuron',
    model_template=nest_model,
    dynamics_params=dynamics_params
)

net.add_edges(
    source=net.nodes(), target=net.nodes(),
    connection_rule=1,
    syn_weight=2.0,
    delay=1.5,
    dynamics_params='ExcToInh.json',
    model_template='static_synapse'
)

net.build()
net.save(output_dir='network_built_in')


virt_exc = NetworkBuilder('virt_exc')
virt_exc.add_nodes(
    N=10,
    model_type='virtual'
)

virt_exc.add_edges(
    target=net.nodes(),
    connection_rule=lambda *_: np.random.randint(0, 10),
    syn_weight=2.0,
    delay=1.0,
    dynamics_params='ExcToInh.json',
    model_template='static_synapse'
)

virt_exc.build()
virt_exc.save(output_dir='network_built_in')

If you have access to an existing model but not the build-script, or if rebuilding the network is too time consuming and/or expensive, then another option to swap out different models is to open the *node_types.csv* in an text editor (especially ones that support csv forats like VSCode or Atom), update the **model_template** and **dynamics_params** attributes, then rerun the simulation.

In [3]:
import pandas as pd

pd.read_csv('network_built_in/net_node_types.csv', sep=' ')

,node_type_id,dynamics_params,model_template,model_type
0,100,custom_model_params.default.json,nest:izhikevich,point_neuron


We encourage you to try running the network yourself by either running in the command line or un
```
$ python run_pointnet.built_in.py config.built_in.json
```
or with the below cell, trying it with different cell models. 

In [4]:
from bmtk.simulator import pointnet

configure = pointnet.Config.from_json('config.built_in.json')
configure.build_env()

network = pointnet.PointNetwork.from_config(configure)
sim = pointnet.PointSimulator.from_config(configure, network)
sim.run()


              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.6.0
 Built: Sep 28 2023 11:52:40

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.

2024-05-02 17:29:54,210 [INFO] Created log file
2024-05-02 17:29:54,225 [INFO] Batch processing nodes for net/0.
2024-05-02 17:29:54,237 [INFO] Batch processing nodes for virt_exc/0.
2024-05-02 17:29:54,263 [INFO] Setting up output directory
2024-05-02 17:29:54,266 [INFO] Building cells.
2024-05-02 17:29:54,276 [INFO] Building recurrent connections
2024-05-02 17:29:54,298 [INFO] Network created.
2024-05-02 17:29:54,301 [INFO] Build virtual cell stimulations for thalamus_spikes
2024-05-02 17:29:54,341 [INFO] Starting Simulation
2024-05-02 17:29:59,093 [INFO] Simulation finished, finalizing results.
2024-05-02 17:30:00,275 [INFO] Done.


One thing to note is that when you replace one cell-model with another you may often get widely varying results. A network that is stable or silent in using one cell type may explode when a model gets replaced. Fixing this may require the or both of the following following:
* Adjusting the **dynamics_params** of the new cell type.
* Adjust the synaptic models and parameters (especially **syn_weight**).

## Example: Importing Custom NESTML cell models

## Example: Overriding the creating of cell models

Before BMTK starts a simulation, it will go ahead and call on NEST to create neurons for all cells using attributes like **model_type**, **model_template**, and **dynamics_params**. BMTK will initialize the creation of these cells for you and for the vast majority of simulations the user does not have to do anythin special other than specify the models and parameters in the SONATA file. 

However, on rare occassions you may find yourself requiring more grainular control of how PointNet initializes and creates the neurons before the simulation begins. For example, you may want to find or alter cell parameters at the start of a simulation. Some models may require a non-standard way of initializing them. Or you may need to attach Parrot or Relay neurons to a subset of your cells. 

When BMTK initializes cells for simualation it will normally call the function `loadNESTModel` function in [`bmtk.simulation.pointnet.default_setters.cell_models`](). This is a simple function that calls that NEST `Create` function to initialize a batch of neurons that share the same model-name and properites.

```python
def loadNESTModel(cell, template_name, dynamics_params):
    return nest.Create(template_name, cell.n_nodes, dynamics_params)
```

If you need to override this function, BMTK provides a special [python decorator](https://peps.python.org/pep-0318/) called `@cell_model`. To use the decorator you can add the following to your *run_pointnet.py* (or in a separate python file imported before you run the simulation) as such

```python
from bmtk.simulator.pointnet import cell_model

@cell_model(directive='nest', model_type='point_neuron')
def loadNESTModel(cell, template_name, dynamics_params):
    nest_nodes = ... # create and initialize cells
    return nest_nodes
```
To explain what is happening in this function
* The `@cell_model` decorator is placed right before our user defined function in order to tell bmtk that this is a special function that will be used in the creation of "nest" cells
  * The parameters tell BMTK that this function will be called for every group of cells with **model_template** of format `nest:...` and have **model_type** of point_neuron
* There are three parameters the BMTK will pass into this function
  * `cell` is copy of cell object and can be used to access all of an cell(s) properities (can be accessed like a dictionary)
  * `template` will be the name of the model (eg. izhikevich, glif_psc, hh_cond, etc)
  * `dynamics_params` will be a dictionary containing the contents of the json or hdf5 **dynamics_params**.
* Inside the function we should create our cell(s), usually using the NEST [Create](https://nest-simulator.readthedocs.io/en/v3.3/ref_material/pynest_apis.html#module-nest.lib.hl_api_nodes) function.
* You must return the results of the Create function, usually of list of ids, back to bmtk.

During the initialization process BMTK will call this custom function. It tries to do so in batches, often initializing cells with the same models and signitures in batches.


<div class="alert alert-block alert-info">
⚠️ WARNING: Overwriting loadNESTModel can be dangerous
</div>

If you're network has many different models then overwritting `loadNESTModel` to deal with only a subset of the cells can have unintended side-effects. However bmtk lets you create specific load methods for different models


#### Overriding Izhikeich model

For example, let's say we want to modify and adjust parameters on-the-fly (eg. we don't want to have to rebuild the network every time) but only for the `izhikevich` models. Specially we want to add jitter to the "d" parameter which won't exists for other model types. To do so we add a function `loadIzhikevich` and in the `@cell_model` parameters make sure that it only applies to `nest:Izhikevich` models, so that all other model types gets created in the default manner.

We add the following function

In [9]:
from bmtk.simulator.pointnet import cell_model
from bmtk.simulator.pointnet.io_tools import io
import nest

@cell_model(directive='nest:izhikevich', model_type='point_neuron')
def loadIzhikevich(cell, template_name, dynamics_params):
    nodes = nest.Create(template_name, cell.n_nodes, dynamics_params)
    
    d_orig = nest.GetStatus(nodes, 'd')
    jitter = cell['jitter']
    d_noisy = d_orig*jitter
    nest.SetStatus(nodes, {'d': d_noisy})
    io.log_info(f'Modifying the parameters of {cell.n_nodes} {template_name} neurons.')
    return nodes

* First we create the cells as normal using the `Create` method. Then we use the `nest.GetStatus` and `nest.SetStatus` to update the "d" parameters for each cell. It is sometimes possible to alter the way `Create()` method is called, but in general it is best to use [`GetStatus`](https://nest-simulator.readthedocs.io/en/v3.3/ref_material/pynest_apis.html#module-nest.lib.hl_api_info) to alter parameters
* The network has a jitter parameter which we can multiple by the default "d" value and is stored in the hdf5 file. We call `cell[jitter]` to get this value as an array, and can use `cell` to fetch other cell attributes.
* As an optional measure we add some logging. This is a good sanity check to make sure our funnction is being called as it should.

After we added our custom function we can load PointNet in the normal manner:

In [10]:
from bmtk.simulator import pointnet

configure = pointnet.Config.from_json('config.iz_updated.json')
configure.build_env()

network = pointnet.PointNetwork.from_config(configure)
sim = pointnet.PointSimulator.from_config(configure, network)
sim.run()

2024-05-02 17:30:46,709 [INFO] Created log file
2024-05-02 17:30:46,719 [INFO] Batch processing nodes for net/0.
2024-05-02 17:30:46,729 [INFO] Batch processing nodes for virt_exc/0.
2024-05-02 17:30:46,748 [INFO] Setting up output directory
2024-05-02 17:30:46,751 [INFO] Building cells.
2024-05-02 17:30:46,755 [INFO] Modifying the parameters of 10 izhikevich neurons.
2024-05-02 17:30:46,757 [INFO] Building recurrent connections
2024-05-02 17:30:46,762 [INFO] Network created.
2024-05-02 17:30:46,764 [INFO] Build virtual cell stimulations for thalamus_spikes
2024-05-02 17:30:46,792 [INFO] Starting Simulation
2024-05-02 17:30:47,388 [INFO] Simulation finished, finalizing results.
2024-05-02 17:30:47,413 [INFO] Done.
